In [1]:
tfidf = preprocessor.TfidfTransformer()
feature_engineering = preprocessor.FeatureTransformer()

y = X["is_duplicate"].values

feature_creator = FeatureUnion([('feature_engineering', feature_engineering), ('tfidf', tfidf)])
clf = LogisticRegression()


#pipe = Pipeline([("features", feature_creator), ("logistic", clf)])
pipe = Pipeline([("features", feature_engineering), ("logistic", clf)])

print pipe.get_params().keys()

NameError: name 'preprocessor' is not defined

In [2]:
#http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [3]:
from scipy.stats import uniform
from time import time


#print cross_val_score(pipe, X, y, scoring="neg_log_loss", cv=skf)
#print cross_val_score(pipe, X, y, scoring="accuracy", cv=skf)

param_dist = {"logistic__C": uniform(0, 5)}
n_iter_search = 20
random_search = RandomizedSearchCV(pipe, param_distributions=param_dist, n_iter=n_iter_search, cv=skf, scoring="neg_log_loss")
random_search.fit(X, y)
              
start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)  

#for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    #y_train, y_test = y[train_index], y[test_index]

NameError: name 'RandomizedSearchCV' is not defined

In [4]:
def save_output(dataset, filename):
    with open (filename, "wb") as handle:
        pickle.dump(dataset, handle)

In [ ]:
output_directory = "../output/models/"
prefix = "first_test_"

save_output(pipe, output_directory+prefix+"logistic_pipeline.pkl") 